In [16]:
import netCDF4 as nc
import numpy as np

# Load the data for both years
#data_2020 = nc.Dataset('/Users/heyj/Desktop/sql project/2020data.nc')
#data_2021 = nc.Dataset('/Users/heyj/Desktop/sql project/2020data.nc')
data_2020 = nc.Dataset('2020data00UTC.nc') #(choose a specific timestamp: 00UTC for each day)
data_2021 = nc.Dataset('2021data00UTC.nc')

# Combine data from both years
z_combined = np.concatenate((data_2020['z'][:], data_2021['z'][:]), axis=0)
u_combined = np.concatenate((data_2020['u'][:], data_2021['u'][:]), axis=0)
v_combined = np.concatenate((data_2020['v'][:], data_2021['v'][:]), axis=0)


In [17]:
# check the dimension
np.info(data_2020['z'][:])


class:  MaskedArray
shape:  (365, 3, 81, 101)
strides:  (196344, 65448, 808, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f1800887010
byteorder:  little
byteswap:  False
type: float64


In [18]:
# check the dimension
np.info(z_combined)

class:  MaskedArray
shape:  (731, 3, 81, 101)
strides:  (196344, 65448, 808, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f16c8ea7010
byteorder:  little
byteswap:  False
type: float64


In [19]:
print("Dimensions:")
for dim in data_2020.dimensions.values():
    print(dim)
    print("\n")


Dimensions:
<class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 101


<class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 81


<class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 3


<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 365




In [20]:
# Preprocessing the data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()

z_normalized = scaler.fit_transform(z_combined.reshape(-1, 1)).reshape(z_combined.shape)
u_normalized = scaler.fit_transform(u_combined.reshape(-1, 1)).reshape(u_combined.shape)
v_normalized = scaler.fit_transform(v_combined.reshape(-1, 1)).reshape(v_combined.shape)

# Combine the parameters to form a single dataset
data_combined = np.stack((z_normalized, u_normalized, v_normalized), axis=-1)

# Pad the data to get even dimensions
padded_data = np.pad(data_combined, ((0, 0), (0, 1), (0, 1), (0, 1), (0, 0)), mode='constant')
np.info(padded_data)

class:  ndarray
shape:  (731, 4, 82, 102, 3)
strides:  (802944, 200736, 2448, 24, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f167615e010
byteorder:  little
byteswap:  False
type: float64


In [21]:
# Preprocessing the data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()

z_normalized = scaler.fit_transform(z_combined.reshape(-1, 1)).reshape(z_combined.shape)
u_normalized = scaler.fit_transform(u_combined.reshape(-1, 1)).reshape(u_combined.shape)
v_normalized = scaler.fit_transform(v_combined.reshape(-1, 1)).reshape(v_combined.shape)

# Combine the parameters to form a single dataset
data_combined = np.stack((z_normalized, u_normalized, v_normalized), axis=-1)

# Pad the data to get even dimensions
padded_data = np.pad(data_combined, ((0, 0), (0, 1), (0, 1), (0, 1), (0, 0)), mode='constant')


# Split the data into training and validation sets
X_train, X_val = train_test_split(padded_data, test_size=0.3, random_state=42)



In [22]:
print("Training data shape", X_train.shape)
print("Validation data shape", X_val.shape)


Training data shape (511, 4, 82, 102, 3)
Validation data shape (220, 4, 82, 102, 3)


In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, UpSampling3D, Add
from tensorflow.keras.models import Model

# Define ResNet block
def resnet_block(input_tensor, filters, kernel_size=(3, 3, 3), strides=(1, 1, 1)):
    x = Conv3D(filters, kernel_size, strides=strides, padding='same')(input_tensor)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    x = Conv3D(filters, kernel_size, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Adjust the shortcut connection
    if strides != (1, 1, 1) or input_tensor.shape[-1] != filters:
        shortcut = Conv3D(filters, (1, 1, 1), strides=strides, padding='same')(input_tensor)
    else:
        shortcut = input_tensor
    
    x = Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    return x


# Define the model architecture
input_shape_padded = (4, 82, 102, 3)
inputs = Input(shape=input_shape_padded)

# Encoder
x = resnet_block(inputs, 32)
x = resnet_block(x, 64)
encoded = resnet_block(x, 128, strides=(2, 2, 2))

# Decoder
x = UpSampling3D((2, 2, 2))(encoded)
x = resnet_block(x, 64)
x = UpSampling3D((1, 1, 1))(x)
x = resnet_block(x, 32)
decoded = Conv3D(3, (3, 3, 3), activation='sigmoid', padding='same')(x)

# Compile the autoencoder
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Print the model summary
autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4, 82, 102,  0                                            
__________________________________________________________________________________________________
conv3d_16 (Conv3D)              (None, 4, 82, 102, 3 2624        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 4, 82, 102, 3 128         conv3d_16[0][0]                  
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 4, 82, 102, 3 0           batch_normalization_10[0][0]     
____________________________________________________________________________________________

In [24]:
# Train the model 
history = autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_val, X_val))


Epoch 1/50
16/16 [==============================] - 28s 1s/step - loss: 0.0185 - val_loss: 0.0124
Epoch 2/50
16/16 [==============================] - 16s 1s/step - loss: 0.0011 - val_loss: 0.0132
Epoch 3/50
16/16 [==============================] - 16s 1s/step - loss: 5.1710e-04 - val_loss: 0.0110
Epoch 4/50
16/16 [==============================] - 16s 1s/step - loss: 3.6860e-04 - val_loss: 0.0081
Epoch 5/50
16/16 [==============================] - 17s 1s/step - loss: 2.9840e-04 - val_loss: 0.0061
Epoch 6/50
16/16 [==============================] - 18s 1s/step - loss: 2.5190e-04 - val_loss: 0.0057
Epoch 7/50
16/16 [==============================] - 19s 1s/step - loss: 2.1882e-04 - val_loss: 0.0050
Epoch 8/50
16/16 [==============================] - 19s 1s/step - loss: 2.8246e-04 - val_loss: 0.0041
Epoch 9/50
16/16 [==============================] - 19s 1s/step - loss: 2.4635e-04 - val_loss: 0.0038
Epoch 10/50
16/16 [==============================] - 20s 1s/step - loss: 2.0945e-04 - val_

In [30]:
autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4, 82, 102,  0                                            
__________________________________________________________________________________________________
conv3d_16 (Conv3D)              (None, 4, 82, 102, 3 2624        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 4, 82, 102, 3 128         conv3d_16[0][0]                  
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 4, 82, 102, 3 0           batch_normalization_10[0][0]     
____________________________________________________________________________________________

In [56]:
# 1. Reconstruction Error:Evaluate how well the autoencoder can reconstruct the input data

from sklearn.metrics import mean_squared_error, mean_absolute_error

decoded_val = autoencoder.predict(X_val)

# Mean Squared Error (MSE)
mse = mean_squared_error(X_val.flatten(), decoded_val.flatten())

# Mean Absolute Error (MAE)
mae = mean_absolute_error(X_val.flatten(), decoded_val.flatten())

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


Mean Squared Error: 0.00010600513860469818
Mean Absolute Error: 0.006431017596301612


In [26]:
# Save Summary text file
original_stdout = sys.stdout  # Save a reference to the original standard output

with open('model_summary.txt', 'w') as f:
    sys.stdout = f  # Change the standard output to the file we created.
    print(autoencoder.summary())
    sys.stdout = original_stdout  # Reset the standard output to its original value




In [27]:
# Save the history object 
import pandas as pd
import pickle

# Convert the history.history dict to a pandas DataFrame
hist_df = pd.DataFrame(history.history)

# Save to csv
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# Save to pickle
with open('history.pkl', 'wb') as file:
    pickle.dump(history.history, file)


In [28]:
# Load the saved history 

loaded_history = pd.read_csv('history.csv')

with open('history.pkl', 'rb') as file:
    loaded_history = pickle.load(file)

In [31]:
# Define the encoder model. Find the low dimension layer according to the model summary. 'activation_15' should be the encoder layer
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('activation_15').output)




In [33]:
encoder_output = autoencoder.get_layer('activation_15').output

In [ ]:
# Save the model
autoencoder.save('autoencoder_yjmodel.h5')

/home/jianhong/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [32]:
# Save the weights
autoencoder.save_weights('autoencoder_yjweights.h5') 

In [34]:
# Save whole training database encoded_database
encoded_database = encoder.predict(X_train)

# Save the encoded data as np and h5py

np. save('encoded_database.npy', encoded_database)

import h5py

with h5py.File('encoded_database', 'w') as h5f:
    h5f.create_dataset('dataset_1', data=encoded_database)

In [ ]:
# Load the model
from tensorflow.keras.models import load_model

autoencoder = load_model('autoencoder_yjmodel.h5')

2023-11-03 13:03:38.158172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [40]:
# Preprocess and Encode the ##Input Day$$ Data, here use 28th Oct 2023 


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Normalize the data
scaler = MinMaxScaler()

Oct_28_2023 = nc.Dataset('Oct_28_00UTC.nc')
#Oct_15_2023 = nc.Dataset('/home/jianhong/Desktop/Analogue-Nowcasting-Model (copy)/15_Oct.nc')
z_input = Oct_28_2023['z'][:]
u_input = Oct_28_2023['u'][:]
v_input = Oct_28_2023['v'][:]

# Normalize the data
scaler = MinMaxScaler()
z_input_normalized = scaler.fit_transform(z_input.reshape(-1, 1)).reshape(z_input.shape)
u_input_normalized = scaler.fit_transform(u_input.reshape(-1, 1)).reshape(u_input.shape)
v_input_normalized = scaler.fit_transform(v_input.reshape(-1, 1)).reshape(v_input.shape)


# Combine the parameters to form a single dataset
data_combined = np.stack((z_input_normalized, u_input_normalized , v_input_normalized), axis=-1)
print(data_combined.shape)
np.info(data_combined)


# Pad the data to get even dimensions
input_day_padded = np.pad(data_combined, ((0,0),(0, 1), (0, 1), (0, 1), (0, 0)), mode='constant')


np.info(input_day_padded)
# Use the encoder to generate the encoded representation of this input day’s data
#input_day_encoded = encoder.predict(input_day_padded)




(1, 3, 81, 101, 3)
class:  ndarray
shape:  (1, 3, 81, 101, 3)
strides:  (589032, 196344, 2424, 24, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x559295c33bf0
byteorder:  little
byteswap:  False
type: float64
class:  ndarray
shape:  (1, 4, 82, 102, 3)
strides:  (802944, 200736, 2448, 24, 8)
itemsize:  8
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x5592961b0a20
byteorder:  little
byteswap:  False
type: float64


In [50]:
encoded_input = encoder.predict(input_day_padded)
np.info(encoded_input)

class:  ndarray
shape:  (1, 2, 41, 51, 128)
strides:  (2141184, 1070592, 26112, 512, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x559299ab2b00
byteorder:  little
byteswap:  False
type: float32


In [49]:
np.info(encoded_database)

class:  ndarray
shape:  (511, 2, 41, 51, 128)
strides:  (2141184, 1070592, 26112, 512, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f14b5914010
byteorder:  little
byteswap:  False
type: float32


In [63]:
# Calculate similarities and find the most similar day:

# Use <Euclidean distances> to calculate the Euclidean distances between the input day's encoded representation and the encoded representations of all the days in the database.



input_flattened = encoded_input.reshape(1,-1)
database_flattened = encoded_database.reshape(encoded_database.shape[0], -1)


#np.info(input_flattened)
#np.info(database_flattened)

from sklearn.metrics.pairwise import euclidean_distances
import numpy as np


# Calculate Euclidean distances 
distances = euclidean_distances(input_flattened, database_flattened)

indices_of_smallest = np.argsort(distances)[:3]

print(indices_of_smallest)
# Calculate Euclidean distances 
#distances = np.array([euclidean(input_flattened,sample) for sample in database_flattened])

#indices_of_smallest = np.argsort(distance)[:3]



[[279 353 134 260 213 364 259  49 144  36 316 436 386 256 484  64 324 485
  117  70 347 314 261 321 339 344  80 257 145 174 215 103 479 295  32 421
  500 307 435 371 182 473 169 401 207  89 322 411 271 202 426  11 404 220
   23 372 291 123 350 342 133 150 507  71 413  12  57  25 282   3  13 227
  108 250 408 196 329 480 264 204 194 390  14  31 265 300 376 381 365  16
  494  83  63 161 331 358  19 410  94 106 368  42 445 385 305  88 244 243
  333 447 453 425 226 354  22  54 170  52 172 304 299 140 375  96 131  29
   21 486 326 200 289 287 387   7 185 187 440 126 232 469 120 346  82 269
  429 430 301 276 181 285 160 463 437   4 356 352 280 392  46  59 306 224
  203 398  72  76 100 195  34  81 217 116 497  77 107 290 422 158 428 441
  155 395 366 137 190 457 193 313 135 503 238 149  87 216  85  84 468  24
  396 247 275 465  97 219 416 317 433 253 508 389  98 510 303 471  35  38
   95  45 148  51 470 492 328 237 233 309 179 167 405 113 363 434 504 493
  432 296 175 343  37 377  17 241 268 

In [ ]:
# 4. Dimensionality Reduction on Encoded Representations (Optional)

from sklearn.manifold import TSNE
import seaborn as sns

# Assuming `encoded_data` contains encoded representations of your entire dataset
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
tsne_results = tsne.fit_transform(encoded_database.reshape(len(encoded_database), -1))

sns.scatterplot(x=tsne_results[:,0], y=tsne_results[:,1], alpha=0.5)
plt.title('t-SNE of Encoded Representations')
plt.show()
